In [1]:
import requests

from bs4 import BeautifulSoup

In [2]:
from urllib.parse import quote, urlparse, urlunparse, urljoin, unquote
from collections import OrderedDict

In [3]:
def urlquote(url):
    url_parsed = urlparse(url)
    
    url_parsed = OrderedDict(
        (k, getattr(url_parsed, k))
        for k in ['scheme', 'netloc', 'path', 'params', 'query', 'fragment'])
    
    for k in ['path', 'params', 'query', 'fragment']:
        url_parsed[k] = quote(url_parsed[k])
    
    return urlunparse(url_parsed.values())

In [4]:
url = 'https://ru.wikipedia.org/wiki/Проект:Знаете_ли_вы/Архив_рубрики/2018-06#19—22_июня'

In [5]:
r = requests.get(urlquote(url))
soup = BeautifulSoup(r.content, 'html.parser')

In [6]:
content_page = soup.find('div', class_='mw-parser-output')

In [7]:
from operator import attrgetter, itemgetter

i_begin = None
content, urls = [], []

for child_i, child in enumerate(content_page.children):
    if child.name == 'h2':
        if child.find('span', class_='mw-headline', id="19—22_июня"):
            i_begin = child_i
        elif i_begin is not None:
            break
    elif i_begin is not None and child.name == 'ul':
        facts = child.find_all('li')
        content.extend(map(attrgetter('text'), facts))
        urls.extend(map(itemgetter('href'), child.find_all('a', href=True)))
        
urls = list(map(lambda u: urljoin('https://ru.wikipedia.org', u), urls))

In [8]:
!mkdir -p dumps

In [9]:
import os

for url in urls:
    filename = unquote(urlparse(url).path)[6:]
    filename = filename.replace(',', '')
    filename = filename.replace('(', '').replace(')', '') + '.html'
    filename = os.path.join("dumps", filename)
    
    r = requests.get(url)
    with open(filename, mode='wb') as f_name:
        f_name.write(r.content)